In [ ]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

'''
For the bootstrap and latest nrt weekly mean obs since 1990 (it was made weekly in Agg_NSIDC_Obs)
filter with a LOESS smoother in time then polynomial fit to get the 
fit parameters. Save the fit parameters since this takes forever.

Later read in fit parameters to extrapolate forward, giving climatological trend 
benchmark

Also use fit parameters for each year to compute an anomaly and save that too for computing alpha

At present this routine is meant to be used once a year, but should make it so that it produces a new climotrend
estimate each week!!!

'''

%matplotlib inline
%load_ext autoreload
%autoreload
import matplotlib
import matplotlib.pyplot as plt
from collections import OrderedDict
import numpy as np
import numpy.ma as ma
import pandas as pd
import struct
import os
import xarray as xr
import glob
import datetime
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
np.seterr(divide='ignore', invalid='ignore')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from esio import EsioData as ed
from esio import ice_plot
from esio import import_data
from esio import metrics
import dask

# General plotting settings
sns.set_style('whitegrid')
sns.set_context("talk", font_scale=.8, rc={"lines.linewidth": 2.5})

# Climatology SIP is average presence for the past 10 years after first computing weekly means starting with Jan 1 each year

In [2]:
# from dask.distributed import Client
# client = Client(n_workers=2)
# client = Client()
# client
dask.config.set(scheduler='threads')  # overwrite default with threaded scheduler
# dask.config.set(scheduler='processes')  # overwrite default with threaded scheduler

In [113]:
# Parameters
today_date = datetime.datetime.now()
nextyear = today_date.year + 1
E = ed.EsioData.load()
mod_dir = E.model_dir
cmod = 'climoSIP'
runType = 'forecast'

In [144]:
#for pred_year in np.arange(1995,2020,1):   # done to make reforecast
for pred_year in [nextyear]:
    start_year = pred_year - 10
    print(start_year,pred_year)  # normally we are computing the fits for predicting far into the future

    #############################################################
    # Load in Data that have already been averaged for each week of the year, always starting on Jan 1
    #############################################################

    # BE SURE THESE ARE NON OVERLAPPING, MUST BE UPDATED FOR NEW DATA
    # Get bootstrap and nrt observations with pole hole already filled in
    ds_81 = xr.open_mfdataset(E.obs['NSIDC_0081']['sipn_nc']+'_yearly_byweek/*byweek.nc', concat_dim='time', autoclose=True, parallel=True).sic
    #ds_51 = xr.open_mfdataset(E.obs['NSIDC_0051']['sipn_nc']+'_yearly_byweek/*byweek.nc', concat_dim='time', autoclose=True, parallel=True)
    ds_79 = xr.open_mfdataset(E.obs['NSIDC_0079']['sipn_nc']+'_yearly_byweek/*byweek.nc', concat_dim='time', autoclose=True, parallel=True).sic

    ds_79=ds_79.sel(time=slice(str(start_year),str(pred_year-1)))  # end year just has to be way in the future
    ds_81=ds_81.sel(time=slice('2015',str(pred_year-1)))  # restrict to before prediciton year, lower year not important

    # Combine bootstrap with NASA NRT
    da_sic = ds_79.combine_first(ds_81)  # takes ds_79 as priority
    ds_79 = None
    ds_81 = None

    #da_sic=ds_81  # for testing
    # add year coordinate
    year_all = [x.year for x in pd.to_datetime(da_sic.time.values)]
    da_sic.coords['year'] = xr.DataArray(year_all, dims='time', coords={'time':da_sic.time})

    # put week coordinate back since combine first rubbed them out
    DOY = [x.timetuple().tm_yday for x in pd.to_datetime(da_sic.time.values)]
    weeks= np.ceil(np.divide(DOY,7))
    weeks = weeks.astype(int)
    da_sic.coords['week'] = xr.DataArray(weeks, dims='time', coords={'time':da_sic.time})
    #print(da_sic)

    # plot so we are sure this is going right
    ocnmask = da_sic.isel(time=-30).notnull()  # take a value near the end when not likely to have missing values
    ocnmask.name = 'oceanmask'

    maxweek = da_sic.sel(time=str(pred_year-1)).week.values[-1]
    print('The last week to compute is ',maxweek)

    # Convert sea ice presence
    ds_sp = (da_sic >= 0.15).astype('int') # This unfortunatly makes all NaN -> zeros...
    ds_sp = ds_sp.where(ocnmask,other=np.nan)  
    ds_sp.coords['week'] = da_sic.week
    #print(ds_sp)

    # Calculate mean SIP
    ds_sip_climo = ds_sp.groupby('week').mean(dim='time')
    ds_sip_climo = ds_sip_climo.sel(week=slice(1,maxweek))

    ds_sip_climo.name = 'SIP'
    ds_sip_climo.coords['time'] = xr.DataArray([datetime.datetime(pred_year,1,1) + datetime.timedelta(days=int(7*(x-1)+3)) for x in np.arange(1,maxweek+1,1)], dims='week')
    ds_sip_climo=ds_sip_climo.swap_dims({'week':'time'})
    print(ds_sip_climo)

    file_out = os.path.join(mod_dir, cmod, runType, 'sipn_nc_yearly_byweek',str(pred_year)+'_byweek.nc')
    ds_sip_climo.to_netcdf(file_out)
    print("Saved",file_out)

2010 2020
The last week to compute is  28
<xarray.DataArray 'SIP' (time: 28, y: 448, x: 304)>
dask.array<shape=(28, 448, 304), dtype=float64, chunksize=(1, 448, 304)>
Coordinates:
  * x        (x) int64 0 1 2 3 4 5 6 7 8 ... 295 296 297 298 299 300 301 302 303
  * y        (y) int64 0 1 2 3 4 5 6 7 8 ... 439 440 441 442 443 444 445 446 447
    lat      (x, y) float64 31.1 31.25 31.4 31.55 ... 34.92 34.77 34.62 34.47
    lon      (x, y) float64 168.3 168.4 168.5 168.7 ... -9.745 -9.872 -9.999
    xm       (x) int64 -3850000 -3825000 -3800000 ... 3675000 3700000 3725000
    ym       (y) int64 5850000 5825000 5800000 ... -5275000 -5300000 -5325000
    week     (time) int64 1 2 3 4 5 6 7 8 9 10 ... 19 20 21 22 23 24 25 26 27 28
  * time     (time) datetime64[ns] 2020-01-04 2020-01-11 ... 2020-07-11


/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6/site-packages/dask/core.py:137: RuntimeWarning: invalid value encountered in greater_equal
  return func(*args2)
/home/disk/sipn/nicway/anaconda3/envs/esio/lib/python3.6/site-packages/dask/array/numpy_compat.py:28: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


Saved /home/disk/sipn/nicway/data/model/climoSIP/forecast/sipn_nc_yearly_byweek/2020_byweek.nc


In [95]:
PlotTest = False
if PlotTest:
    tmpsip1=ds_sip_climo.sel(week=1) # save one time at random for plot verification
    tmpsip2=ds_sip_climo.sel(week=maxweek) # save one time at random for plot verification

    # plot one time at random to ensure it is about right Nplots has to be one more than you'd think
    (f, axes) = ice_plot.multi_polar_axis(ncols=2, nrows=1, Nplots = 3, sizefcter=3)
    tmpsip1.plot.pcolormesh(cmap='Reds',ax=axes[0], x='lon', y='lat',transform=ccrs.PlateCarree())
    tmpsip2.plot.pcolormesh(cmap='Reds',ax=axes[1], x='lon', y='lat',transform=ccrs.PlateCarree())

In [163]:
# Hardcoded start date (makes incremental weeks always the same)
start_t = datetime.datetime(1950, 1, 1) # datetime.datetime(1950, 1, 1)
# Params for this plot
Ndays = 7 # time period to aggregate maps to (default is 7)

init_start_date = np.datetime64('2018-01-01') # first date we have computed metrics
                   
#init_start_date = np.datetime64('2019-01-01') # speeds up substantially b

cd = today_date +  datetime.timedelta(days=365)

init_slice = np.arange(start_t, cd, datetime.timedelta(days=Ndays)).astype('datetime64[ns]')
# init_slice = init_slice[-Npers:] # Select only the last Npers of periods (weeks) since current date
init_slice = init_slice[init_slice>=init_start_date] # Select only the inits after init_start_date

init_midpoint = np.arange(start_t- datetime.timedelta(days=3), cd- datetime.timedelta(days=3), datetime.timedelta(days=Ndays)).astype('datetime64[ns]')
init_midpoint = init_midpoint[init_midpoint>=init_start_date] # Select only the inits after init_start_date

print('init_slices are at end of the range')
print(init_slice[0],init_slice[-1])
print('shift to midpoint of range')
print(init_midpoint[0],init_midpoint[-1])


init_slices are at end of the range
2018-01-07T00:00:00.000000000 2020-07-05T00:00:00.000000000
2018-01-04T00:00:00.000000000 2020-07-02T00:00:00.000000000


In [177]:
files = os.path.join(mod_dir, cmod, runType, 'sipn_nc_yearly_byweek','*_byweek.nc')
da = xr.open_mfdataset(files, concat_dim='time', parallel=True).SIP
cd = today_date +  datetime.timedelta(days=375)
da=da.sel(time=slice(str(init_start_date),str(cd)))  # end year just has to be way in the future
da=da.chunk({'time': 132, 'y': 8})
print(da)
dar=da.interp(time=init_midpoint,method= 'linear')
dar['time']=init_slice
dar = dar.drop('week')
print(dar)
file_out = os.path.join(mod_dir, cmod, runType, 'interpolated','climoSIP_2018_to_nextyear.nc')
dar.to_netcdf(file_out)
print("Saved",file_out)

<xarray.DataArray 'SIP' (time: 132, y: 448, x: 304)>
dask.array<shape=(132, 448, 304), dtype=float64, chunksize=(132, 8, 304)>
Coordinates:
  * x        (x) int64 0 1 2 3 4 5 6 7 8 ... 295 296 297 298 299 300 301 302 303
  * y        (y) int64 0 1 2 3 4 5 6 7 8 ... 439 440 441 442 443 444 445 446 447
    lat      (x, y) float64 dask.array<shape=(304, 448), chunksize=(304, 8)>
    lon      (x, y) float64 dask.array<shape=(304, 448), chunksize=(304, 8)>
    xm       (x) int64 dask.array<shape=(304,), chunksize=(304,)>
    ym       (y) int64 dask.array<shape=(448,), chunksize=(8,)>
    week     (time) int64 dask.array<shape=(132,), chunksize=(132,)>
  * time     (time) datetime64[ns] 2018-01-04 2018-01-11 ... 2020-07-11
<xarray.DataArray 'SIP' (time: 131, y: 448, x: 304)>
dask.array<shape=(131, 448, 304), dtype=float64, chunksize=(131, 8, 304)>
Coordinates:
  * x        (x) int64 0 1 2 3 4 5 6 7 8 ... 295 296 297 298 299 300 301 302 303
  * y        (y) int64 0 1 2 3 4 5 6 7 8 ... 439 440

In [176]:
PlotTest = False
if PlotTest:
    tmpsip1=da.isel(time=80) # save one time at random for plot verification
    tmpsip2=dar.isel(time=80) # save one time at random for plot verification
    tmpsip2=tmpsip2-tmpsip1

    # plot one time at random to ensure it is about right Nplots has to be one more than you'd think
    (f, axes) = ice_plot.multi_polar_axis(ncols=2, nrows=1, Nplots = 3, sizefcter=3)
    tmpsip1.plot.pcolormesh(cmap='Reds',ax=axes[0], x='lon', y='lat',transform=ccrs.PlateCarree())
    tmpsip2.plot.pcolormesh(cmap='Reds',ax=axes[1], x='lon', y='lat',transform=ccrs.PlateCarree())